In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import time
import os

In [ ]:
from utils.cub_config import Config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = Config.DATA_DIR
num_classes = 200
batch_size = 16
num_epochs = 15
learning_rate = 1e-5

In [3]:
print(f"Using device: {device}")

Using device: cuda


In [6]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(448),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(448),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [8]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Substituir a última camada totalmente conectada
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

for param in model.parameters():
    param.requires_grad = True


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 186MB/s] 


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch+1}/{num_epochs}")
        print("-" * 30)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                dataloader = train_loader
            else:
                model.eval()
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'val':
                scheduler.step(epoch_loss)

                # Salvar melhor modelo
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    if Config.SAVE_MODEL_DIR is not None:
                        save_path = os.path.join(Config.SAVE_MODEL_DIR, "best_resnet50_cub_fullfinetune.pth")
                    else:
                        save_path = "best_resnet50_cub_fullfinetune.pth"

                    torch.save(model.state_dict(), save_path)

    print(f"\nTreinamento concluído! Melhor acurácia de validação: {best_acc:.4f}")

In [14]:
train_model(model, criterion, optimizer, scheduler, num_epochs=num_epochs)


Época 1/15
------------------------------
train Loss: 5.2680 Acc: 0.0137
val Loss: 5.1889 Acc: 0.0341

Época 2/15
------------------------------
train Loss: 5.0156 Acc: 0.0621
val Loss: 4.4410 Acc: 0.1807

Época 3/15
------------------------------
train Loss: 4.3338 Acc: 0.1603
val Loss: 3.4292 Acc: 0.3117

Época 4/15
------------------------------
train Loss: 3.7279 Acc: 0.2806
val Loss: 2.8060 Acc: 0.4402

Época 5/15
------------------------------
train Loss: 3.2951 Acc: 0.3652
val Loss: 2.3672 Acc: 0.5215

Época 6/15
------------------------------
train Loss: 2.9216 Acc: 0.4396
val Loss: 1.9904 Acc: 0.5876

Época 7/15
------------------------------
train Loss: 2.6282 Acc: 0.4867
val Loss: 1.7101 Acc: 0.6398

Época 8/15
------------------------------
train Loss: 2.3794 Acc: 0.5314
val Loss: 1.5105 Acc: 0.6828

Época 9/15
------------------------------
train Loss: 2.1926 Acc: 0.5703
val Loss: 1.3748 Acc: 0.7043

Época 10/15
------------------------------
train Loss: 2.0353 Acc: 0.595